In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from pandas import read_csv, DataFrame

## Config Vars

In [9]:
conf_test_dataset_path = 'data/test_dataset.csv'
shift_predictions_path = 'data/shift_predictions'
model_type = 'lstm'
conf_dataset_columns_to_drop = ["@timestamp"]
shifts = 5

## Load Dataset

In [22]:
train_dataset = read_csv(conf_test_dataset_path , header=0, index_col=0)
train_dataset.drop(conf_dataset_columns_to_drop, axis=1, inplace=True)
train_dataset[train_dataset.columns] = StandardScaler().fit_transform(train_dataset)
train_dataset_no_anotation = train_dataset.drop("anomaly", axis=1)

In [25]:
train_dataset = read_csv(conf_test_dataset_path , header=0, index_col=0)

## Load Predctions

In [11]:
shift_predictions = []
for shift in range(1, shifts+1):
    dset = read_csv(shift_predictions_path + '/predicted_data_' + model_type + '_model_' + str(shift) + '.csv' , header=0, index_col=0)
    shift_predictions.append(dset)

## Calculating error vector (Using MSE)
e (t) = [e<sub>11</sub><sup>(t)</sup>, ...,e<sub>1l</sub><sup>(t)</sup>, ...,e<sub>d1</sub><sup>(t)</sup>, ...,e<sub>dl</sub><sup>(t)</sup>]

In [12]:
error_vects = DataFrame(columns = [1,2,3,4,5])
for row in range(5, train_dataset.shape[0]):
    vect = []
    for shift in range(shifts):
        error = mean_squared_error(shift_predictions[shift].iloc[[row - (shift + 1)]],train_dataset_no_anotation.iloc[[row]])
        vect.append(error)
    error_vects.loc[row] = vect 
    

In [27]:
error_vects.index = train_dataset.index[5:]
error_vects[["anomaly"]] = train_dataset[["anomaly"]][5:]

In [28]:
error_vects.to_csv("data/error_vects.csv")